<a href="https://colab.research.google.com/github/gjhernandez1234/sibe-IA-LLM/blob/main/Generando%20LLM%20basado%20en%20los%20datos%20bibliograficos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install tiktoken
!pip install -U sentence-transformers

!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util

from datasets import concatenate_datasets

import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datasets import Dataset

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:
df = pd.read_csv('/content/biblio_original.csv')

dataset = Dataset.from_pandas(df)

In [ ]:
dataset

Dataset({
    features: ['biblionumber', 'titulo', 'abstract_esp', 'abstract_eng', 'abstract_tipos', 'personal_name', 'terminos', 'terminos_ingles'],
    num_rows: 17497
})

In [ ]:
# Filtrar los resúmenes que están en español (donde 'abstract_esp' no es nulo)
subset_esp = dataset.filter(lambda x: x['abstract_esp'] is not None)

# Filtrar los resúmenes que están en inglés (donde 'abstract_eng' no es nulo)
subset_eng = dataset.filter(lambda x: x['abstract_eng'] is not None and x['terminos_ingles'] is not None)

# Filtrar los resúmenes que están en inglés (donde 'abstract_eng' no es nulo y no tiene terminos en ingles solo en español)
subset_2_eng = dataset.filter(lambda x: x['abstract_eng'] is not None and x['terminos_ingles'] is None and x['abstract_esp'] is None)

Filter:   0%|          | 0/17497 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17497 [00:00<?, ? examples/s]

Filter:   0%|          | 0/17497 [00:00<?, ? examples/s]

In [ ]:
subset_esp

Dataset({
    features: ['biblionumber', 'titulo', 'abstract_esp', 'abstract_eng', 'abstract_tipos', 'personal_name', 'terminos', 'terminos_ingles'],
    num_rows: 10348
})

In [ ]:
subset_eng

Dataset({
    features: ['biblionumber', 'titulo', 'abstract_esp', 'abstract_eng', 'abstract_tipos', 'personal_name', 'terminos', 'terminos_ingles'],
    num_rows: 5162
})

In [ ]:
subset_2_eng

Dataset({
    features: ['biblionumber', 'titulo', 'abstract_esp', 'abstract_eng', 'abstract_tipos', 'personal_name', 'terminos', 'terminos_ingles'],
    num_rows: 4178
})

In [ ]:
subset_2_eng[1:2]

{'biblionumber': [25],
 'titulo': ['Plant molecular biology: a laboratory manual'],
 'abstract_esp': [None],
 'abstract_eng': ['This manual covers the whole range of molecular biology techniques, genetic engineering as well as cytogenetics of plants. Each chapter starts with an introduction into the basic approach followed by detailed methods sections with easy-to-follow protocols and comprehensive troubleshooting. The first part of the book introduces basic molecular methodology such as DNA extraction, blotting, production of libraries and RNA cloning. The second part describes analytical approaches, in particular RAPD and RFLP, while the final part encompasses a variety of gene transfer techniques and both molecular and cytological analysis. The manual will be of great use to both the first-timer and the experienced scientist.'],
 'abstract_tipos': ['eng'],
 'personal_name': ['Clark, Melody S.::::editor'],
 'terminos': ['Biología molecular de las plantas | Genética molecular de las p

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
import re
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short

def remove_accents(input_str):
    # Convierte los caracteres con acentos a su forma base sin acentos
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return "".join([char for char in nfkd_form if not unicodedata.combining(char)])

def clean_text(sentence_batch, language='abstract_esp'):
    # Extrae las listas de la entrada (batch) usando el nombre de la columna dinámica
    #text_list = sentence_batch[language]
    title_list = sentence_batch['titulo']

    # Condicional para elegir los términos según el idioma
    if language == 'abstract_esp':
        text_list = sentence_batch[language]
        terms_list = sentence_batch['terminos']
    elif language == 'abstract_eng':
        text_list = sentence_batch[language]
        terms_list = sentence_batch['terminos_ingles']
    else:
        text_list = sentence_batch['abstract_eng']
        terms_list = sentence_batch['terminos']




    # Crear las listas para almacenar los resultados procesados
    cleaned_titles = []
    cleaned_terms = []
    cleaned_texts = []
    cleaned_abstract = []

    # Iterar sobre todas las listas a la vez
    for text, title, terms in zip(text_list, title_list, terms_list):
        # Convierte el texto a minúsculas
        text = text.lower()

        # Elimina acentos
        text = remove_accents(text)
        #terms_list = "".join([str(term) for term in terms_list if term]) if isinstance(terms_list, list) else terms_list
        #terms_list = remove_accents(terms_list) if terms_list else ""
        # Verifica si terms es NaN o None y aplica remove_accents o asigna cadena vacía
        terms = remove_accents(terms) if pd.notna(terms) else ""
        terms = terms.replace(' | ', ',')

        # Elimina URLs
        text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

        # Elimina menciones y hashtags
        text = re.sub(r'\@\w+|\#\w+', '', text)

        # Elimina puntuación, números y palabras cortas
        text = strip_punctuation(text)
        text = strip_numeric(text)
        text = strip_short(text, minsize=2)

        # Elimina las palabras comunes (stop words) según el idioma
        if language == 'abstract_esp':
            stop_words = set(stopwords.words('spanish'))
        elif language == 'abstract_eng':
            stop_words = set(stopwords.words('english'))
        else:
            stop_words = set(stopwords.words('english'))

        # Tokeniza el texto y filtra stopwords
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stop_words]

        # Une las palabras filtradas
        filtered_text = ' '.join(filtered_text)

        # Agrega los resultados procesados a las listas correspondientes
        cleaned_titles.append(title)
        cleaned_terms.append(terms)
        cleaned_texts.append(filtered_text)
        cleaned_abstract.append(text)

    # Devuelve un diccionario donde las claves son las columnas
    return {
        'titulo_x': cleaned_titles,
        'terminos_x': cleaned_terms,
        'abstract_x': cleaned_abstract,
        'text': cleaned_texts
    }


In [ ]:
# Mapear la función clean_text a los abstracts en español
sentences_corpus_esp = subset_esp.map(lambda batch: clean_text(batch, 'abstract_esp'), batched=True)


Parameter 'function'=<function <lambda> at 0x785dc4558b80> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/10348 [00:00<?, ? examples/s]

In [ ]:

# Mapear la función clean_text a los abstracts en inglés
sentences_corpus_eng = subset_eng.map(lambda batch: clean_text(batch, 'abstract_eng'), batched=True)

Map:   0%|          | 0/5162 [00:00<?, ? examples/s]

In [ ]:

# Mapear la función clean_text a los abstracts en inglés
sentences_corpus_eng_2 = subset_2_eng.map(lambda batch: clean_text(batch, 'indistinto'), batched=True)

Map:   0%|          | 0/4178 [00:00<?, ? examples/s]

In [ ]:
sentences_corpus_eng_2[1:2]

{'biblionumber': [25],
 'titulo': ['Plant molecular biology: a laboratory manual'],
 'abstract_esp': [None],
 'abstract_eng': ['This manual covers the whole range of molecular biology techniques, genetic engineering as well as cytogenetics of plants. Each chapter starts with an introduction into the basic approach followed by detailed methods sections with easy-to-follow protocols and comprehensive troubleshooting. The first part of the book introduces basic molecular methodology such as DNA extraction, blotting, production of libraries and RNA cloning. The second part describes analytical approaches, in particular RAPD and RFLP, while the final part encompasses a variety of gene transfer techniques and both molecular and cytological analysis. The manual will be of great use to both the first-timer and the experienced scientist.'],
 'abstract_tipos': ['eng'],
 'personal_name': ['Clark, Melody S.::::editor'],
 'terminos': ['Biología molecular de las plantas | Genética molecular de las p

In [ ]:
sentences_corpus_esp.num_rows

10348

In [ ]:
sentences_corpus_eng.num_rows

5162

In [ ]:
sentences_corpus_esp[1]

{'biblionumber': 8,
 'titulo': 'Programa de manejo de la Reserva de la Biosfera Mariposa Monarca, México: ',
 'abstract_esp': 'La región de la Mariposa Monarca es una zona prioritaria para el diseño de políticas ambientales, se ubica en un territorio que abarca parte de los estados de México y Michoacán, es heredera de un importante patrimonio histórico y cultural, muchos de los conflictos para alcanzar la sustentabilidad se expresan aquí y es uno de los temas que llaman la atención de la sociedad: se trata de una zona indígena y campesina pobre con una vasta biodiversidad. Su importancia crece debido a que en ella se presenta el fenómeno natural de la migración de una de las especies que compartimos los socios comerciales de América del Norte, Canadá, Estados Unidos y México, lo que le hace estar simbólicamente ligada al Tratado de Libre Comercio y constituye uno de los asuntos globales de interés para diversas naciones, por lo que los ojos internacionales están puestos en la región.'

## GENERANDO EL SET DE DATOS DE PRUEBAS

In [ ]:
dataset_test_biblio_esp = sentences_corpus_esp.shuffle(seed=42).select(range(10))
dataset_test_biblio_eng = sentences_corpus_eng.shuffle(seed=42).select(range(10))
dataset_test_biblio_eng_spa = sentences_corpus_eng_2.shuffle(seed=42).select(range(10))

# Combinar los tres datasets
dataset_test_combinado = concatenate_datasets([dataset_test_biblio_esp, dataset_test_biblio_eng, dataset_test_biblio_eng_spa])

# Revolver el dataset combinado
dataset_test_combinado = dataset_test_combinado.shuffle(seed=42)

# Guardar el dataset combinado en un archivo CSV
#dataset_test_combinado.to_csv("dataset_test_combinado.csv", index=True)


In [ ]:
dataset_test_combinado

Dataset({
    features: ['biblionumber', 'titulo', 'abstract_esp', 'abstract_eng', 'abstract_tipos', 'personal_name', 'terminos', 'terminos_ingles', 'titulo_x', 'terminos_x', 'abstract_x', 'text'],
    num_rows: 30
})

# GENERANDO TRANSFORMERS Y EMBBEDDINGS

In [ ]:
# Instalar las librerías necesarias
!pip install transformers torch

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

# Cargar el modelo y el tokenizer de T5
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Función para procesar cada batch
def process_batch(batch):
    texts = batch['text']
    target_texts = batch['terminos_x']

    # Inicializar listas para almacenar tensores
    embeddings = []
    num_tokens_list = []
    text_original_len_list = []
    text_tokenized_len_list = []
    target_token_ids = []
    input_ids_list = []  # Lista para guardar tensores de input_ids
    target_inputs_list = []  # Lista para guardar tensores de target input_ids
    attention_masks_list = []  # Lista para guardar los attention masks
    labels_list = []  # Lista para labels

    # Procesar cada texto en el batch
    for text, target_text in zip(texts, target_texts):
        # Tokenizar el texto
        inputs = tokenizer(text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
        tokens_ids = inputs['input_ids'][0]

        # Tokenizar el target_text (términos)
        target_inputs = tokenizer(target_text, return_tensors="pt", padding='max_length', truncation=True, max_length=256)
        target_ids = target_inputs['input_ids'][0]

        #with torch.no_grad():
        #    outputs = model.encoder(input_ids=inputs['input_ids'])  # Solo se usa el encoder
         #   last_hidden_states = outputs.last_hidden_state
         #   embedding = last_hidden_states.mean(dim=1).squeeze()

        # Guardar los tensores directamente
        #embeddings.append(embedding)
        input_ids_list.append(inputs['input_ids'].squeeze())  # Guarda los input_ids como tensor
        target_inputs_list.append(target_inputs['input_ids'].squeeze())  # Guarda los target_inputs como tensor
        attention_masks_list.append(inputs['attention_mask'].squeeze())  # Guarda los attention masks
        labels_list.append(target_inputs['input_ids'].squeeze())  # Guarda los labels (input_ids del target)

        # Otros datos
        num_tokens_list.append(len(tokens_ids))
        text_original_len_list.append(len(text))
        text_tokenized_len_list.append(len(tokenizer.decode(tokens_ids, skip_special_tokens=True)))
        target_token_ids.append(target_ids)

    # Convertir listas a tensores usando padding
    input_ids_tensor = torch.nn.utils.rnn.pad_sequence(input_ids_list, batch_first=True)  # Apilar input_ids
    target_inputs_tensor = torch.nn.utils.rnn.pad_sequence(target_inputs_list, batch_first=True)  # Apilar target_inputs
    attention_mask_tensor = torch.nn.utils.rnn.pad_sequence(attention_masks_list, batch_first=True)  # Apilar attention masks
    labels_tensor = torch.nn.utils.rnn.pad_sequence(labels_list, batch_first=True)  # Apilar labels

    # Devolver los tensores y otros datos
    return {
        'num_tokens': num_tokens_list,
        'text_original_len': text_original_len_list,
        'text_tokenized_len': text_tokenized_len_list,
        'target_token_ids': target_token_ids,  # Añadir los token IDs de los términos
        'input_ids': input_ids_tensor,  # Devolver input_ids como tensor
        'target_inputs': target_inputs_tensor,  # Devolver target_inputs como tensor
        'attention_mask': attention_mask_tensor,  # Devolver attention_mask como tensor
        'labels': labels_tensor  # Devolver labels como tensor
    }

# Ejemplo de uso con un subset de tu dataset
# subset_first_10 = subset_esp.select(range(10))
# sentences_corpus_esp = subset_first_10.map(lambda batch: process_batch(batch), batched=True, with_tqdm=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# ENTRENANDO

## Terminos en español

In [ ]:
# Tomar solo las primeras 10 filas del dataset
subset_first_10 = sentences_corpus_esp  #.select(range(10))

# Aplicar la función a las primeras 10 filas del dataset
subset_first_10 = subset_first_10.map(lambda batch: process_batch(batch), batched=True)


Map:   0%|          | 0/10348 [00:00<?, ? examples/s]

## Terminos en Ingles

In [ ]:
# Tomar solo las primeras 10 filas del dataset
subset_first_10_eng = sentences_corpus_eng #.select(range(10))

# Aplicar la función a las primeras 10 filas del dataset
subset_first_10_eng = subset_first_10_eng.map(lambda batch: process_batch(batch), batched=True)

Map:   0%|          | 0/5162 [00:00<?, ? examples/s]

## Terminos en Español donde el Abstract esta en ingles


In [ ]:
# Tomar solo las primeras 10 filas del dataset
subset_first_10_eng_2 = sentences_corpus_eng_2 #.select(range(10))

# Aplicar la función a las primeras 10 filas del dataset
subset_first_10_eng_2 = subset_first_10_eng_2.map(lambda batch: process_batch(batch), batched=True)

Map:   0%|          | 0/4178 [00:00<?, ? examples/s]

## Generando subsets para entrenamiento en ESPAÑOL (80% y 20%)

In [ ]:
# Asegúrate de que 'subset_esp' contiene tu dataset ya procesado
# Dividimos el conjunto de datos en entrenamiento y evaluación (80% entrenamiento, 20% evaluación)
train_test_split = subset_first_10.train_test_split(test_size=0.2, seed=42)  # Establecer una semilla para reproducibilidad

# Acceder a los conjuntos de datos de entrenamiento y evaluación
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

# Imprimir el tamaño de los conjuntos de datos para verificar
print(f'Tamaño del conjunto de entrenamiento en español: {len(train_dataset)}')
print(f'Tamaño del conjunto de evaluación en español: {len(eval_dataset)}')


Tamaño del conjunto de entrenamiento en español: 8278
Tamaño del conjunto de evaluación en español: 2070


## Generando subsets para entrenamiento en INGLES (80% y 20%)

In [ ]:
# Asegúrate de que 'subset_esp' contiene tu dataset ya procesado
# Dividimos el conjunto de datos en entrenamiento y evaluación (80% entrenamiento, 20% evaluación)
train_test_split_eng = subset_first_10_eng.train_test_split(test_size=0.2, seed=42)  # Establecer una semilla para reproducibilidad

# Acceder a los conjuntos de datos de entrenamiento y evaluación
train_dataset_eng = train_test_split_eng['train']
eval_dataset_eng = train_test_split_eng['test']

# Imprimir el tamaño de los conjuntos de datos para verificar
print(f'Tamaño del conjunto de entrenamiento en ingles: {len(train_dataset_eng)}')
print(f'Tamaño del conjunto de evaluación en ingles: {len(eval_dataset_eng)}')




Tamaño del conjunto de entrenamiento en ingles: 4129
Tamaño del conjunto de evaluación en ingles: 1033


#Generando subsets para entrenamiento en INGLES-ESPAÑOL (80% y 20%)

In [ ]:
# Asegúrate de que 'subset_esp' contiene tu dataset ya procesado
# Dividimos el conjunto de datos en entrenamiento y evaluación (80% entrenamiento, 20% evaluación)
train_test_split_eng_spa = subset_first_10_eng_2.train_test_split(test_size=0.2, seed=42)  # Establecer una semilla para reproducibilidad

# Acceder a los conjuntos de datos de entrenamiento y evaluación
train_dataset_eng_spa = train_test_split_eng_spa['train']
eval_dataset_eng_spa = train_test_split_eng_spa['test']

# Imprimir el tamaño de los conjuntos de datos para verificar
print(f'Tamaño del conjunto de entrenamiento en ingles-español: {len(train_dataset_eng_spa)}')
print(f'Tamaño del conjunto de evaluación en ingles-español: {len(eval_dataset_eng_spa)}')

Tamaño del conjunto de entrenamiento en ingles-español: 3342
Tamaño del conjunto de evaluación en ingles-español: 836


## Combinamos los sets de entrenamiento y de test

In [ ]:
# Concatenar los conjuntos de entrenamiento
combined_train_dataset = concatenate_datasets([train_dataset, train_dataset_eng, train_dataset_eng_spa])

# Concatenar los conjuntos de prueba
combined_eval_dataset = concatenate_datasets([eval_dataset, eval_dataset_eng, eval_dataset_eng_spa])

# Mostrar información sobre los nuevos datasets
print(f"Combined Train Dataset: {combined_train_dataset}")
print(f"Combined Eval Dataset: {combined_eval_dataset}")

Combined Train Dataset: Dataset({
    features: ['biblionumber', 'titulo', 'abstract_esp', 'abstract_eng', 'abstract_tipos', 'personal_name', 'terminos', 'terminos_ingles', 'titulo_x', 'terminos_x', 'abstract_x', 'text', 'num_tokens', 'text_original_len', 'text_tokenized_len', 'target_token_ids', 'input_ids', 'target_inputs', 'attention_mask', 'labels'],
    num_rows: 15749
})
Combined Eval Dataset: Dataset({
    features: ['biblionumber', 'titulo', 'abstract_esp', 'abstract_eng', 'abstract_tipos', 'personal_name', 'terminos', 'terminos_ingles', 'titulo_x', 'terminos_x', 'abstract_x', 'text', 'num_tokens', 'text_original_len', 'text_tokenized_len', 'target_token_ids', 'input_ids', 'target_inputs', 'attention_mask', 'labels'],
    num_rows: 3939
})


## Proceso de entrenamiento

In [ ]:
# Cargar el modelo y el tokenizer de T5
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
from transformers import EarlyStoppingCallback
import os
os.environ["WANDB_DISABLED"] = "true"

# Actualizamos los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",  # Evaluamos al final de cada steps
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,  # Cargar el mejor modelo al final del entrenamiento
    metric_for_best_model="eval_loss",  # Métrica para determinar el mejor modelo
    greater_is_better=False,  # Indica que una menor pérdida es mejor
)

# Definimos el Trainer con conjunto de entrenamiento y evaluación
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=combined_train_dataset,
    eval_dataset=combined_eval_dataset,
    tokenizer=tokenizer,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]  # Detener si no hay mejora en 2 épocas

)

# Entrenamos el modelo
#trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


# GUARDANDO MODELO

In [ ]:
import os
import datetime

# Obtener la fecha actual
fecha_actual = datetime.datetime.now()
nombre_modelo = "/content/t5_finetuned"
# Verificar si la carpeta ya existe
if not os.path.exists(nombre_modelo):
    # Si la carpeta no existe, crea la carpeta con el nombre y fecha
    nombre_modelo = f"/content/t5_finetuned_{fecha_actual.year}_{fecha_actual.strftime('%j')}"




# Guardar el modelo y el tokenizador en la carpeta generada dinámicamente
model.save_pretrained(nombre_modelo)
tokenizer.save_pretrained(nombre_modelo)

('/content/t5_finetuned/tokenizer_config.json',
 '/content/t5_finetuned/special_tokens_map.json',
 '/content/t5_finetuned/spiece.model',
 '/content/t5_finetuned/added_tokens.json')

In [ ]:
nombre_modelo

'/content/t5_finetuned'

In [ ]:
from google.colab import files
import shutil

# Comprimir el modelo en un archivo zip
shutil.make_archive(nombre_modelo, 'zip', nombre_modelo)

# Descargar el archivo zip
files.download(nombre_modelo + '.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
nombre_modelo

'/content/t5_finetuned'

# Probando modelo


In [ ]:
import pandas as pd
from datasets import Dataset
datos_prueba = pd.read_csv('/content/dataset_test_combinado.csv')

dataset_test_combinado = Dataset.from_pandas(datos_prueba)

In [ ]:
dataset_test_combinado

Dataset({
    features: ['biblionumber', 'titulo', 'abstract_esp', 'abstract_eng', 'abstract_tipos', 'personal_name', 'terminos', 'terminos_ingles', 'titulo_x', 'terminos_x', 'abstract_x', 'text'],
    num_rows: 40
})

In [ ]:
import os
import datetime

# Obtener la fecha actual
fecha_actual = datetime.datetime.now()
nombre_modelo = "/content/t5_finetuned"

# Verificar si la carpeta ya existe
if not os.path.exists(nombre_modelo):
    # Si la carpeta no existe, crea la carpeta con el nombre y fecha
    nombre_modelo = f"/content/t5_finetuned_{fecha_actual.year}_{fecha_actual.strftime('%j')}"

#nombre_modelo='Voicelab/vlt5-base-keywords'

from transformers import T5Tokenizer, T5ForConditionalGeneration

# Cargar el modelo y el tokenizador entrenados
model = T5ForConditionalGeneration.from_pretrained(nombre_modelo)
tokenizer = T5Tokenizer.from_pretrained(nombre_modelo)

# Función para generar palabras clave
def generate_keywords(batch):
    # Concatenar título y abstract como entrada para el modelo
    input_texts = [f"Keywords: {title} {abstract}" for title, abstract in zip(batch['titulo'], batch['abstract_x'])]
    # Tokenizar las entradas
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generar predicciones
    outputs = model.generate(**inputs, max_length=512, no_repeat_ngram_size=3, num_beams=5)

    # Decodificar las predicciones a texto
    keywords_list = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

    # Limpiar las salidas y asegurarse de que sean listas separadas por comas
    keywords_cleaned = [', '.join([keyword.strip() for keyword in keywords.split(',') if keyword.strip()]) for keywords in keywords_list]

    return {'terminos_generados_ia': keywords_cleaned}




In [ ]:
nombre_modelo

'/content/t5_finetuned'

In [ ]:
# Aplicar la función a cada batch del Dataset

# Tomar solo las primeras 10 filas del dataset
#dataset  = subset_first_10.select(range(20))
dataset  = dataset_test_combinado #.select(range(3))

dataset = dataset.map(generate_keywords, batched=True)

# Mostrar un subset del Dataset con la nueva columna
# Convertir a DataFrame para visualizar fácilmente
result_df = dataset.to_pandas()

# Muestra el DataFrame resultante con las columnas deseadas
result_df[['titulo', 'abstract_x', 'terminos', 'terminos_generados_ia']].head(2)

Map:   0%|          | 0/40 [00:00<?, ? examples/s]

,titulo,abstract_x,terminos,terminos_generados_ia
0,A Barrier to our shared environment: the borde...,the objective of this work is to contribute to...,Evaluación del impacto ambiental | Fronteras |...,Keywords: A Barrier to our shared environment:...
1,New symbiotic associations of Hyperiid amphipo...,hyperiid amphipods are holoplanktonic marine c...,Anfípodos hiperídeos | Zooplancton marino | Si...,Keywords: Hyperiid amphipods (Peracarida) and ...


In [ ]:
#result_df[['biblionumber','titulo', 'abstract_x', 'text', 'terminos_x', 'terminos_generados_ia']].to_csv('/content/outputs_test.csv', index=True)

In [ ]:
tesauro_df = pd.read_csv('/content/autoridades_origin.csv')

datasetAuth = Dataset.from_pandas(tesauro_df)

In [ ]:
datasetAuth

Dataset({
    features: ['authid', 'nombre', 'cancelados', 'terminos_cancelados', 'authtypecode', 'Unnamed: 5'],
    num_rows: 11961
})

In [ ]:
!pip install fuzzywuzzy[speedup]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.2 MB/s eta 0:00:00


In [ ]:
from fuzzywuzzy import process, fuzz
import pandas as pd

# Umbral de similitud para considerar una coincidencia (ajustable)
umbral_similitud = 85

# Crear un diccionario para relacionar cada término cancelado con su nombre
terminos_dict = {}
for _, row in tesauro_df.iterrows():
    if pd.notna(row['nombre']):
        # Combina `nombre` y `terminos_cancelados`, separa y limpia cada término
        terminos_completos = [row['nombre'].strip().lower()]
        if pd.notna(row['terminos_cancelados']):
            terminos_cancelados = [t.strip().lower() for t in row['terminos_cancelados'].split(" | ") if t.strip()]
            terminos_completos.extend(terminos_cancelados)

        # Asocia cada término con su respectivo nombre
        for term in terminos_completos:
            terminos_dict[term] = row['nombre'].strip()

# Función para comparar términos generados con el tesauro y devolver el nombre correcto
def comparar_con_tesauro(terminos_generados):
    coincidencias = []
    # Crear lista de términos generados, limpiando y convirtiendo a minúsculas
    terminos_generados_list = [t.strip().lower().replace("keywords:", "") for t in terminos_generados.split(",") if t.strip()]

    for termino in terminos_generados_list:
        # Buscar la mejor coincidencia en `terminos_dict` usando fuzzy matching
        matches = process.extractOne(termino, terminos_dict.keys(), scorer=fuzz.ratio, score_cutoff=umbral_similitud)

        if matches:
            coincidencias.append(terminos_dict[matches[0]])  # Agregar el término `nombre` del tesauro
        else:
            coincidencias.append("No localizada")  # Marcar como "No localizada" si no hay coincidencias

    return coincidencias

# Aplicar la función al DataFrame `result_df`
result_df['coincidencias_tesauro'] = result_df['terminos_generados_ia'].apply(comparar_con_tesauro)

# Calcular el porcentaje de coincidencias
def calcular_porcentaje(coincidencias):
    num_coincidencias = sum(1 for termino in coincidencias if termino != "No localizada")
    total_terminos = len(coincidencias)
    return (num_coincidencias / total_terminos) * 100 if total_terminos > 0 else 0

result_df['porcentaje_coincidencias'] = result_df['coincidencias_tesauro'].apply(calcular_porcentaje)



# Función para calcular el porcentaje de coincidencias entre terminos_x y coincidencias_tesauro
def calcular_porcentaje_coincidencias_x(row):
    # Verificar si terminos_x es None o vacío
    if pd.isna(row['terminos']) or row['terminos'].strip() == "":
        return 0  # Si es None o vacío, retornar 0

    terminos_x_list = [t.strip().lower() for t in row['terminos'].split(" | ") if t.strip()]
    coincidencias_list = [t.strip().lower() for t in row['coincidencias_tesauro'] if t.strip() != "No localizada"]

    # Contar coincidencias
    num_coincidencias = sum(1 for termino in terminos_x_list if termino in coincidencias_list)
    total_terminos_x = len(terminos_x_list)

    return (num_coincidencias / total_terminos_x) * 100 if total_terminos_x > 0 else 0

# Aplicar la función al DataFrame
result_df['porcentaje_coincidencias_x_human'] = result_df.apply(calcular_porcentaje_coincidencias_x, axis=1)


# Agregar una nueva columna que cuente el número de palabras en coincidencias_tesauro
result_df['num_palabras_coincidencias'] = result_df['coincidencias_tesauro'].apply(
    lambda x: sum(len(termino.split()) for termino in x if termino != "No localizada")
)



# Ver los resultados
result_df[['terminos_generados_ia','terminos', 'coincidencias_tesauro', 'porcentaje_coincidencias','porcentaje_coincidencias_x_human','num_palabras_coincidencias']].head(5)


,terminos_generados_ia,terminos,coincidencias_tesauro,porcentaje_coincidencias,porcentaje_coincidencias_x_human,num_palabras_coincidencias
0,Keywords: A Barrier to our shared environment:...,Evaluación del impacto ambiental | Fronteras |...,"[No localizada, No localizada]",0.000000,0.000000,0
1,Keywords: Hyperiid amphipods (Peracarida) and ...,Anfípodos hiperídeos | Zooplancton marino | Si...,[No localizada],0.000000,0.000000,0
2,"keywords: Cafetal, Agroecosistema de cafetal, ...",Plantas comestibles | Conocimiento tradicional...,"[Cafetal, No localizada, No localizada]",33.333333,16.666667,1
3,"Keywords: Infrasound, Data processing, Nuclear...",Nuclear arms control | Underground nuclear exp...,"[No localizada, Procesamiento de datos, No loc...",20.000000,0.000000,3
4,Keywords: The ecology of plants,Plantas | Ecología vegetal,[No localizada],0.000000,0.000000,0


In [ ]:
result_df[['biblionumber','titulo', 'abstract_x', 'terminos_x','terminos_generados_ia','num_palabras_coincidencias','coincidencias_tesauro','porcentaje_coincidencias','porcentaje_coincidencias_x_human']].to_csv('/content/outputs_test.csv', index=True)